In [70]:
import numpy as np
import pandas as pd
from typing import NamedTuple
import math
from collections import Counter
from typing import TypeVar
from typing import Dict
from collections import defaultdict
from scratch.linear_algebra import Vector
import random

In [2]:
# =======================
# Import data 
# =========================
df=pd.read_csv('drug200.csv')
print(f'columns\t:{df.shape[1]} \nrows\t:{df.shape[0]}')


columns	:6 
rows	:200


In [3]:
df.columns=df.columns.str.lower()
df_=df[:]
df_.head()

,age,sex,bp,cholesterol,na_to_k,drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [11]:
# scaling data
column =['age','na_to_k']
for col in column:
    vals=[]
    min_=df_[col].min()
    max_=df_[col].max()
    for i in range(len(df['age'])):
        val=(df_.loc[i,col]-min_)/(max_-min_)
        vals.append(val)
    df_[col]=vals

In [12]:
df_.head()

,age,sex,bp,cholesterol,na_to_k,drug
0,0.135593,F,HIGH,HIGH,0.596848,DrugY
1,0.542373,M,LOW,HIGH,0.213397,drugC
2,0.542373,M,LOW,HIGH,0.120239,drugC
3,0.220339,F,NORMAL,HIGH,0.047814,drugX
4,0.779661,F,LOW,HIGH,0.368191,DrugY


In [13]:
# =======================
# cleaning data
# =======================

df_.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          200 non-null    float64
 1   sex          200 non-null    object 
 2   bp           200 non-null    object 
 3   cholesterol  200 non-null    object 
 4   na_to_k      200 non-null    float64
 5   drug         200 non-null    object 
dtypes: float64(2), object(4)
memory usage: 9.5+ KB


In [14]:
df_.isnull().sum()

age            0
sex            0
bp             0
cholesterol    0
na_to_k        0
drug           0
dtype: int64

In [15]:
df_.describe()

,age,na_to_k
count,200.000000,200.000000
mean,0.496864,0.306945
std,0.280412,0.225904
min,0.000000,0.000000
25%,0.271186,0.130605
50%,0.508475,0.239774
75%,0.728814,0.410001
max,1.000000,1.000000


In [16]:
df.columns.tolist()

['age', 'sex', 'bp', 'cholesterol', 'na_to_k', 'drug']

In [17]:
# Cek data with object type
for i in df_.columns.tolist():
    print(i)
    print (type(df_.loc[0,i]))
    


age
<class 'numpy.float64'>
sex
<class 'str'>
bp
<class 'str'>
cholesterol
<class 'str'>
na_to_k
<class 'numpy.float64'>
drug
<class 'str'>


In [18]:
data =df_[:].values.tolist()

In [94]:
class candidate(NamedTuple):
    age :int
    sex :str
    bp:str
    cholesterol:str
    na_to_k:float
    drug :int

inputs =[]
    

In [54]:
type(inputs[0])

__main__.candidate

In [95]:
for i in df_.values:
    input =candidate(*i)
    inputs.append(input)

In [96]:
def shuffel(indx:list,train_prob:float):
    data_test =int(train_prob*len(indx))
    random.shuffle(indx)
    return indx[data_test:],indx[:data_test]

def split_data(input:list,test_prob:float):
    indx=[i for i in range(len(input))]
    train_prob=1-test_prob
    idx_test,idx_train =shuffel(indx,train_prob)
    return (
        [input[i] for i in idx_train],
        [input[i] for i in idx_test]
    )
    

In [ ]:
# Build

def prob (labels=list[any]):
    totals =len(labels)
    return [count/totals for count in Counter(labels).values() ]

def entropy( labels:list[any]):
    return sum(-p*math.log(p,2) for p in prob (labels) if p>0)
    
def partition_entropy (subsets:list[any]):
    total=sum(len(subset) for subset in subsets)
    return sum (entropy (subset)*len(subset)/total for subset in subsets)


In [22]:
# make partition from data
T =TypeVar('T')
def partition_by(input:list[T],attribute:str):
    partition:Dict[any,list[T]] =defaultdict(list)
    for input in inputs:
        key=getattr(input,attribute)
        partition[key].append(input)
    return partition

def partition_entropy_by(inputs: list[T],atribute:str,label_atribute:str):
    partitions =partition_by(inputs,atribute)
    labels = [[getattr (input,label_atribute) for input in partition] for partition in partitions.values()]
    return partition_entropy(labels)

In [23]:
partition_entropy_by(inputs,'age','sex')

0.7619964840063298

In [24]:
# Build tree
from typing import Union,NamedTuple,Any,Dict
class leaf(NamedTuple):
    value :Any

class split(NamedTuple):
    atribute :str
    subtrees:Dict
    default_value:Any =None

Desicion_Tree=Union[leaf,split]


In [25]:
def classify(tree:Desicion_Tree, input :Any ):
    if isinstance (tree,leaf):
        return tree.value
    substree_key= getattr (input,tree.atribute)
    if substree_key not in tree.subtrees:
        return tree.default_value
    subtree=tree.subtrees[substree_key]
    return classify(subtree,input)


    

In [26]:
def build_tree(inputs:list[Any],split_atribute:list[str],target_atribute:str):
    label_counts=Counter(getattr( input,target_atribute) for input in inputs)
    most_common_labels=label_counts.most_common(1)[0][0]
    if len(label_counts)==1:
        return leaf(most_common_labels)
    
    if not split_atribute:
        return  leaf(most_common_labels)
        
    def split_entropy_func(atribute: str):
        return partition_entropy_by(inputs,atribute,target_atribute)
    

    best_atribute=min(split_atribute, key=split_entropy_func)    
    partitions=partition_by(inputs,best_atribute)
    new_attribute=[a for a in split_atribute if a!= best_atribute]

    subtrees ={attribute_value:build_tree(subset,new_attribute,target_atribute)
               for attribute_value, subset in partitions.items()}
    return split(best_atribute,subtrees,default_value=most_common_labels)

In [111]:
df_train,df_test=split_data(inputs,0.25)

In [ ]:
# build tree
tree=build_tree(inputs,['age', 'sex', 'bp', 'cholesterol', 'na_to_k'],'drug')

In [ ]:
# cek data with df_test
true_=0
for data in df_test:
    predict =classify(tree,data)
    real=data[-1]
    if predict==real:
        true_=true_+1

accuracy=true_/len(df_test) 

In [110]:
print (f"accuracy: {accuracy}")

accuracy: 0.98
